In [1]:
import umls_tables_processing, utils
import datetime

#### Previous coding related to processing of MRREL table: elaboration of relationships and evaluation of which is more convenient

In [2]:
_ = umls_tables_processing.count_relationships(rel_type = 'RELA')

844
0:00:48


This is the number of all the relationships `RELA` in the whole UMLS: they correspond to the particular relationships

In [3]:
umls_tables_processing.count_relationships(rel_type = 'REL')

10
0:00:47


{'AQ', 'CHD', 'PAR', 'QB', 'RB', 'RN', 'RO', 'RQ', 'SIB', 'SY'}

These are the `REL` relationships and the total number: they correspond to generical relationships

The following lines tell us which are the relationship which link the 256 elements of the seed with the concept `COPD`: this is an evolution of the original seed building method.

In [2]:
double_rel = umls_tables_processing.concepts_related_to_concept(two_way = True, polishing_rels = True, switch_key= 'con')

In [5]:
single_rel = umls_tables_processing.concepts_related_to_concept(two_way = False, polishing_rels = True, switch_key= 'con')

In [12]:
single_rel

{'C0024117': ['see_from',
  'was_a',
  'see',
  'has_entry_version',
  'mapped_to',
  'has_expanded_form',
  'inverse_isa',
  'entry_version_of',
  'possibly_equivalent_to',
  'isa',
  'use',
  'used_for',
  'inverse_was_a',
  'mapped_from',
  'expanded_form_of',
  'same_as'],
 'C1527303': [],
 'C0040233': ['contraindicated_with_disease'],
 'C0041030': ['contraindicated_with_disease'],
 'C0087093': ['contraindicated_with_disease'],
 'C0724707': ['contraindicated_with_disease'],
 'C0965618': ['may_treat'],
 'C0982282': ['contraindicated_with_disease'],
 'C2699757': ['may_treat'],
 'C2699758': ['may_treat'],
 'C2983887': ['contraindicated_with_disease'],
 'C4519232': ['may_treat'],
 'C0005768': [],
 'C0007807': [],
 'C0007994': [],
 'C0008903': [],
 'C0009678': [],
 'C0012160': [],
 'C0013217': [],
 'C0013557': [],
 'C0013943': [],
 'C0014445': [],
 'C0014508': [],
 'C0015033': [],
 'C0015127': [],
 'C0017399': [],
 'C0019665': [],
 'C0025520': [],
 'C0025953': [],
 'C0026566': [],
 'C00

The two lines distinguish the both ways relationships from the one way.

#### Double ways

In [7]:
_ = set([i for j in double_rel.values() for i in j])
print(_)
print(len(_))

{'mapped_from', 'answer_to', 'subset_includes_concept', 'associated_finding_of', 'replaces', 'manifestation_of', 'related_to', 'fragments_for_synonyms_of', 'focus_of', 'gene_associated_with_disease', 'replaced_by', 'may_treat', 'same_as', 'has_fragments_for_synonyms', 'gene_product_malfunction_associated_with_disease', 'used_for', 'has_focus', 'classified_as', 'has_course', 'cdrh_parent_of', 'has_finding_site', 'has_contraindicated_drug', 'inverse_was_a', 'clinical_course_of', 'has_clinical_course', 'classifies', 'see', 'was_a', 'disease_has_associated_anatomic_site', 'is_associated_anatomic_site_of', 'see_from', 'has_manifestation', 'contraindicated_with_disease', 'has_associated_finding', 'has_cdrh_parent', 'has_answer', 'associated_with_malfunction_of_gene_product', 'associated_morphology_of', 'mapped_to', 'has_expanded_form', 'finding_site_of', 'entry_version_of', 'inverse_isa', 'has_entry_version', 'course_of', 'concept_in_subset', 'expanded_form_of', 'disease_has_associated_gene'

#### One way

In [8]:
_ = set([i for j in single_rel.values() for i in j])
print(_)
print(len(_))

{'mapped_from', 'subset_includes_concept', 'related_to', 'gene_associated_with_disease', 'replaced_by', 'may_treat', 'same_as', 'has_fragments_for_synonyms', 'gene_product_malfunction_associated_with_disease', 'used_for', 'has_focus', 'classified_as', 'cdrh_parent_of', 'inverse_was_a', 'clinical_course_of', 'see', 'was_a', 'see_from', 'is_associated_anatomic_site_of', 'has_manifestation', 'contraindicated_with_disease', 'has_associated_finding', 'has_cdrh_parent', 'has_answer', 'associated_morphology_of', 'mapped_to', 'has_expanded_form', 'entry_version_of', 'finding_site_of', 'inverse_isa', 'has_entry_version', 'course_of', 'expanded_form_of', 'use', 'isa', 'possibly_equivalent_to'}
36


#### Building the seed for analogic evaluation: we obtain lists of CUIs for each relationship in which they're involved with COPD

In [16]:
seed_analog_both = umls_tables_processing.concepts_related_to_concept(two_way = True, switch_key = 'rel')
print(len(seed_analog_both))
seed_analog_both

53


defaultdict(list,
            {'isa': ['C0001883',
              'C0024115',
              'C0024115',
              'C0006267',
              'C0008677',
              'C0034067',
              'C0155883',
              'C0006272',
              'C0024117',
              'C0034067',
              'C0155874',
              'C0264371',
              'C0340044',
              'C0348693',
              'C0348693',
              'C0348818',
              'C0730604',
              'C0730605',
              'C0730607',
              'C1277261',
              'C3662842',
              'C3714496',
              'C4038730',
              'C4040148',
              'C0264220',
              'C0264364',
              'C0425442',
              'C0578508',
              'C0600260',
              'C0746102'],
             'related_to': ['C0004096',
              'C0008677',
              'C0024115',
              'C0004096',
              'C0008677',
              'C0024115',
              'C0034067'

In [17]:
seed_analog_one = umls_tables_processing.concepts_related_to_concept(two_way = False, switch_key = 'rel')
print(len(seed_analog_one))
seed_analog_one

36


defaultdict(list,
            {'see_from': ['C0024117'],
             'entry_version_of': ['C0024117',
              'C0024117',
              'C0024117',
              'C0024117'],
             'contraindicated_with_disease': ['C0040233',
              'C0041030',
              'C0087093',
              'C0724707',
              'C0982282',
              'C2983887'],
             'may_treat': ['C0965618', 'C2699757', 'C2699758', 'C4519232'],
             'mapped_to': ['C1847014', 'C1854729', 'C1969833', 'C0024117'],
             'see': ['C0024117', 'C0600260'],
             'use': ['C0024117', 'C0024117', 'C4551465'],
             'isa': ['C0006267',
              'C0008677',
              'C0034067',
              'C0155883',
              'C0006272',
              'C0024117',
              'C0034067',
              'C0155874',
              'C0264371',
              'C0340044',
              'C0348693',
              'C0348693',
              'C0348818',
              'C0730604',
  

In counting the number of RELA relationships, the second approach return one more relation: it is due to the presence of the emtpy relation `''`

Importing the dictionary for augmenting the seed with labels (preferred and not) from MRCONSO table and the vocabulary of a w2v embedding (PubMed-and-PMC-w2v was chosen)

In [8]:
#dict_conso = umls_tables_processing.cui_strings()
#utils.inputs_save(dict_conso, 'dict_conso')
dict_conso = utils.inputs_load('Utilities/dict_conso')
vocab = utils.inputs_load('Utilities/PubMed-and-PMC-w2v-vocab')

0:00:12
0:00:00


#### For optimizing the computation, the seed labels out-of-vocabulary are discarded 

Extraction of the 256 CUIs of seed and augmentation with all the labels

In [10]:
copd_cuis = list(single_rel.keys())
seed_rel, _ = umls_tables_processing.extracting_strings(copd_cuis, dict_conso)

0:00:00


In [11]:
seed_rel

{'C0024117': ['Chronic Obstructive Airway Disease',
  'Chronic obstructive airway disease',
  'Chronic obstructive airways disease NOS',
  'Chronic Obstructive Airways Disease',
  'Chronic airway obstruction',
  'Chronic Obstructive Lung Disease',
  'CHRONIC OBSTRUCTIVE LUNG DISEASE',
  'Chronic obstructive lung disease',
  'chronic obstructive lung disease',
  'Chronic obstructive lung disease, NOS',
  'Pulmonary Disease, Chronic Obstructive',
  'CHRONIC OBSTRUCTIVE PULMONARY DISEASE',
  'chronic obstructive pulmonary disease',
  'Chronic obstructive pulmonary disease',
  'PULMONARY DISEASE, CHRONIC OBSTRUCTIVE',
  'Chronic obstructive pulmonary disease NOS',
  'Chronic Obstructive Pulmonary Diseases',
  'Chronic Obstructive Pulmonary Disease',
  'COLD',
  'COPD',
  'COPD NOS',
  'HYPERACTIVE AIRWAY DISEASE',
  'Chronic airflow limitation',
  'chronic airflow limitation',
  'COAD - Chronic obstructive airways disease',
  'Chronic airway disease',
  'chronic airway disease',
  'chronic

Discarding oov labels per CUI

In [19]:
new_dict = umls_tables_processing.discarding_labels_oov(vocab, seed_rel)    

0%
0:00:04
1%
0:00:05
2%
0:00:06
3%
0:00:07
3%
0:00:08
4%
0:00:09
5%
0:00:09
6%
0:00:09
7%
0:00:10
7%
0:00:10
8%
0:00:10
9%
0:00:10
10%
0:00:11
10%
0:00:11
11%
0:00:11
12%
0:00:11
13%
0:00:12
14%
0:00:12
14%
0:00:13
15%
0:00:13
16%
0:00:13
17%
0:00:13
17%
0:00:13
18%
0:00:14
19%
0:00:17
20%
0:00:19
21%
0:00:19
21%
0:00:22
22%
0:00:27
23%
0:00:31
24%
0:00:34
25%
0:00:36
25%
0:00:37
26%
0:00:41
27%
0:00:44
28%
0:00:47
28%
0:00:50
29%
0:00:50
30%
0:00:52
31%
0:00:53
32%
0:00:56
32%
0:00:57
33%
0:00:57
34%
0:00:58
35%
0:00:58
35%
0:00:58
36%
0:00:58
37%
0:01:00
38%
0:01:01
39%
0:01:03
39%
0:01:08
40%
0:01:09
41%
0:01:10
42%
0:01:10
42%
0:01:11
43%
0:01:16
44%
0:01:17
45%
0:01:19
46%
0:01:21
46%
0:01:27
47%
0:01:28
48%
0:01:31
49%
0:01:34
50%
0:01:36
50%
0:01:39
51%
0:01:42
52%
0:01:47
53%
0:01:52
53%
0:01:54
54%
0:01:59
55%
0:02:01
56%
0:02:03
57%
0:02:07
57%
0:02:13
58%
0:02:18
59%
0:02:22
60%
0:02:24
60%
0:02:28
61%
0:02:29
62%
0:02:36
63%
0:02:39
64%
0:02:44
64%
0:02:47
65%
0:02:50
66%


In [21]:
print(str(len(new_dict))+'/'+str(len(copd_cuis)))

120/256


Notice: the number of seed elements is lower than usual. The OOVs have been deleted

In [23]:
seed_analog_polish = umls_tables_processing.seed_analogy_labels(seed_analog_one, new_dict)

We obtained a dictionary of dictionaries: the first key is represented by a relation. For each relation a dictionary with a CUI as key and a list of labels (preferred and not) as value

The idea would be make a data structure of all the elements linked with the same relations seen in `COPD`. 

We started with all the pairs by relation:

In [28]:
#t = datetime.datetime.now().replace(microsecond=0)
#seed_pairs = {}
#list_rels = list(seed_analog_polish.keys())
#list_rels.remove('')
#seed_pairs = umls_tables_processing.count_pairs(list_rels)
#print(datetime.datetime.now().replace(microsecond=0)-t) 

36
0:01:03
0:01:03


For avoiding the building of dictionary, it is locally saved and loaded

The duplicated pairs (if exist) for each relation are discarded.

In [31]:
seed_pairs = utils.inputs_load('Utilities/pairs_for_relation')
seed_pairs = {k:list(set(v)) for k,v in seed_pairs.items()}

A list of all involved CUIs is built

In [33]:
asda = set()
for k,v in seed_pairs.items():
    tmp = [item for sublist in v for item in sublist]
    asda.update(tmp)
    print('Relation ' +k+ ' is done')

all_cuis_involved = list(asda)

Relation entry_version_of is done
Relation mapped_to is done
Relation has_entry_version is done
Relation inverse_isa is done
Relation same_as is done
Relation see_from is done
Relation see is done
Relation mapped_from is done
Relation isa is done
Relation used_for is done
Relation replaced_by is done
Relation subset_includes_concept is done
Relation use is done
Relation has_fragments_for_synonyms is done
Relation has_expanded_form is done
Relation expanded_form_of is done
Relation has_answer is done
Relation possibly_equivalent_to is done
Relation contraindicated_with_disease is done
Relation finding_site_of is done
Relation cdrh_parent_of is done
Relation has_manifestation is done
Relation has_associated_finding is done
Relation associated_morphology_of is done
Relation may_treat is done
Relation classified_as is done
Relation related_to is done
Relation was_a is done
Relation inverse_was_a is done
Relation has_focus is done
Relation has_cdrh_parent is done
Relation clinical_course_of

In [35]:
len(all_cuis_involved)

1340127

The number of all the CUIs having same relations of `COPD`concepts is very high

A dictionary with all the involved CUIs and the correspondent labels (preferred and not) is built

In [36]:
all_labels_involved, _ = umls_tables_processing.extracting_strings(all_cuis_involved, dict_conso)

0:00:02


In [38]:
len([i for k, v in all_labels_involved.items() for i in v])

4489335

In [39]:
len([i for k, v in all_labels_involved.items() for i in v])/len(all_cuis_involved)

3.3499325063967818

For each CUI there are ~4 labels, raising even more the already high number of concepts: at this point discarding OOV elements is a key-operation

With the `discarding_labels_oov` method is impossible proceed

In [ ]:
all_labels_involved = umls_tables_processing.discarding_labels_oov(vocab, all_labels_involved)

and even with a lambda function the process employs more than 10 days. And the actual measure of analogy still missing, where a confrontation between `COPD` seed labels and this huge one has to be performed

In [41]:
a = datetime.datetime.now().replace(microsecond=0)
g = [list(filter(lambda x: x in vocab, sublist)) for sublist in list(all_labels_involved.values())[:100]]
print(datetime.datetime.now().replace(microsecond=0)-a)

0:00:22


In [42]:
(len([i for k, v in all_labels_involved.items() for i in v])*(22/100))/3600

274.34825